In [65]:
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
import requests
import re

In [66]:
url = "https://goodreads.com/book/show/1001053.Between_Two_Fires"
authors = {}

In [67]:
df = pd.read_csv('../csv/goodreads_clean.csv').dropna()
authors_desc = []


In [68]:
counter = 0
def getAuthorDescription(link):
    global counter
    
    try:

        print('Getting reviews from ' + str(link))
        r = requests.get(str(link))
        soup = BeautifulSoup(r.text, 'html.parser')
        req = soup.find('div', {'id':'bookAuthors'})
        if not req: return

        auth = req.select('a', {'class':'authroName'})

        for i in auth:
            new_each = re.search(r'href="(.*?)"',str(i))

            if not new_each: continue

            start = new_each.span()[0]
            end = new_each.span()[1]
            part = re.sub("href=\"", "", str(i)[start:end])
            final = re.sub("\"", "", part)
            
            if final == "#": continue
        
            author_url = urllib.request.urlopen(final)
            soup2 = BeautifulSoup(author_url, 'html.parser')
            name = soup2.find('h1', {'class':'authorName'}).text
            name = re.sub("\n", "", str(name))
            x = soup2.find('div', {'class':'aboutAuthorInfo'}).select('span',{'id':'freeTextauthor'})
            
            counter = counter + 1
            
            if not x:
                authors_desc.append([name, ""])
                return
            elif len(x) == 1: 
                x = x[0]
                x = re.sub(r'.*<span id="freeTextauthor.*?">', "", str(x))
                x = re.sub(r'</span>', "", str(x))
                x = re.sub(r'<.*?>', '', str(x))
                authors_desc.append([name, str(x)])
            else: 
                x = x[1]
                x = re.sub(r'.*<span id="freeTextauthor.*?">', "", str(x))
                x = re.sub(r'</span>', "", str(x))
                x = re.sub(r'<.*?>', '', str(x))
                authors_desc.append([name, str(x)])
     

    except Exception as e:
        print(str(e) + " -> " + url)

    if counter % 4 == 0:
        df1 = pd.DataFrame(authors_desc, columns = ['Author', 'Description'])
        df1.to_csv('authors.csv', encoding='utf-8', index=False)

    return True

In [69]:
df['link'].apply(getAuthorDescription)


Getting reviews from https://goodreads.com/book/show/1001053.Between_Two_Fires
Getting reviews from https://goodreads.com/book/show/10010552-fashion-sourcebook-1920s
Getting reviews from https://goodreads.com/book/show/1001077.Hungary_56
Getting reviews from https://goodreads.com/book/show/1001079.All_American_Anarchist
Getting reviews from https://goodreads.com/book/show/1001090.The_Human_Equation
Getting reviews from https://goodreads.com/book/show/1001092.Competitive_Advantage_Through_People
Getting reviews from https://goodreads.com/book/show/1001126.Hawaii
Getting reviews from https://goodreads.com/book/show/100114.Genuine_Happiness
Getting reviews from https://goodreads.com/book/show/1001204.Anthropological_Studies_of_Religion
Getting reviews from https://goodreads.com/book/show/1001209.Anarchism_And_Ecology
Getting reviews from https://goodreads.com/book/show/100120.Happiness
Getting reviews from https://goodreads.com/book/show/1001220.Anarchism_Environmental_Survival
Getting re

In [ ]:

#df1.to_csv('authors.csv', encoding='utf-8', index=False)